# 学习使用elastic search

In [2]:
from elasticsearch import Elasticsearch

# 实例化es
es = Elasticsearch(hosts="http://localhost:9200")
# 创建一个索引为test的索引
es.indices.create(index='test')

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'test'})

## 插入数据

In [3]:
# 在test索引id为1的位置插入一条数据
es.index(index='test',
         id=1,
         body={"id":'1',"name":"小明"})

doc = {
    'name': '方天',
    'age': '23'
}
# 在test索引id为2的位置插入一条数据
es.index(index='test', id='2', document=doc)

ObjectApiResponse({'_index': 'test', '_type': '_doc', '_id': '2', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1})

In [4]:
doc = {
    'name': '杨晨',
    'age': '22'
}
# 在test索引id为3的位置插入一条数据
es.create(index='test',id='3', document=doc)

ObjectApiResponse({'_index': 'test', '_type': '_doc', '_id': '3', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1})

## 删除数据

In [6]:
# 删除test索引id为3的数据
es.delete(index='test',id='3')

ObjectApiResponse({'_index': 'test', '_type': '_doc', '_id': '3', '_version': 2, 'result': 'deleted', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1})

## 更新数据

In [10]:
doc = {
    'name': '李邱俊',
    'age': '20',
    'new_field': 'new_value'
}
# 将test索引id为2的数据更新为新数据
es.update(index='test',id='2',doc=doc)

ObjectApiResponse({'_index': 'test', '_type': '_doc', '_id': '2', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 5, '_primary_term': 1})

## 查询操作

In [11]:
# 查询test索引id为1的数据
response = es.get(index='test',id='2')

# 解析响应
index = response['_index']
id = response['_id']
source = response['_source']  # 这是实际的文档数据

print(f"Index: {index}")
print(f"ID: {id}")
print(f"Source: {source}")

Index: test
ID: 2
Source: {'name': '李邱俊', 'age': '20', 'new_field': 'new_value'}


In [12]:
# 只显示test索引的age字段信息
print(es.search(index='test',filter_path=['hits.hits._source.age']))

{'hits': {'hits': [{'_source': {'age': '20'}}]}}


In [13]:
# 查询test索引中，从序号为1的位置查询两条数据
print(es.search(index='test', from_=1, size=2))

{'took': 5, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'test', '_type': '_doc', '_id': '1', '_score': 1.0, '_source': {'id': '1', 'name': '小明'}}]}}


In [14]:
# 查询test索引中，name为杨晨或小明的数据
print(es.search(index='test', query={'terms':{'name.keyword':['杨晨','小明']}}))

{'took': 14, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'test', '_type': '_doc', '_id': '1', '_score': 1.0, '_source': {'id': '1', 'name': '小明'}}]}}


In [16]:
# 通配符查询 查询test索引中，name字段为小*的数据
print(es.search(index='test',query={'wildcard':{'name.keyword':'小?'}}))

{'took': 4, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'test', '_type': '_doc', '_id': '1', '_score': 1.0, '_source': {'id': '1', 'name': '小明'}}]}}


In [17]:
# 正则查询
print(es.search(index='test',query={'regexp':{'name.keyword':'李.*'}}))

{'took': 5, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'test', '_type': '_doc', '_id': '2', '_score': 1.0, '_source': {'name': '李邱俊', 'age': '20', 'new_field': 'new_value'}}]}}


## 删除索引

In [18]:
# 删除索引为test的索引
es.indices.delete(index='test')

ObjectApiResponse({'acknowledged': True})